<a href="https://colab.research.google.com/github/AlienMedoff/4.01/blob/main/1201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 0. ЧИСТОЕ СКАЧИВАНИЕ С football-data.co.uk
# ==========================================
import pandas as pd, requests, os
from tqdm import tqdm

# какие сезоны и лиги нужны
SEASONS = ['2122','2223','2324','2425']   # 2021-22 … 2024-25 (5 сезонов)
LEAGUES = {                                 # код на сайте : короткое имя
    'E0':'Premier-League','D1':'Bundesliga','I1':'Serie-A','SP1':'LaLiga','F1':'Ligue-1',
    'CL':'Champions-League','EL':'Europa-League','EC':'Conference-League',
    'WC22':'World-Cup-2022','NU23':'Nations-League-2023','EUQ24':'Euro-2024-qualifiers'
}
BASE_URL = 'https://www.football-data.co.uk/mmz4281/{}/{}.csv'
OUT_CSV  = '/content/BIG_SHKOLA_DATASET.csv'

all_df = []
for yr in tqdm(SEASONS, desc='Сезоны'):
    for code, name in LEAGUES.items():
        url = BASE_URL.format(yr, code)
        try:
            # проверим, доступен ли файл
            r = requests.get(url, timeout=15)
            if r.status_code != 200:
                continue
            df = pd.read_csv(url)
            # оставляем нужные колонки и переименовываем
            cols = {'HomeTeam':'home','AwayTeam':'away','FTHG':'FTHG','FTAG':'FTAG',
                    'B365H':'B365H','B365D':'B365D','B365A':'B365A'}
            df = df[list(cols.keys())].rename(columns=cols)
            df['league'] = name
            df['season'] = f'20{yr[:2]}-20{yr[2:]}'
            all_df.append(df)
        except Exception as e:
            # пропускаем несуществующие комбинации
            continue

if not all_df:
    raise RuntimeError('Не удалось загрузить ни одного файла – проверьте сезоны/коды лиг на сайте.')

big = pd.concat(all_df, ignore_index=True)
big = big.dropna(subset=['B365H','B365D','B365A'])
big.to_csv(OUT_CSV, index=False)
print(f'✅ Сохранено {len(big)} матчей в {OUT_CSV}')

Сезоны: 100%|██████████| 4/4 [00:50<00:00, 12.61s/it]

✅ Сохранено 8810 матчей в /content/BIG_SHKOLA_DATASET.csv


In [ ]:
# ==========================================
# 0. УСТАНОВКИ
# ==========================================
import pandas as pd, numpy as np, requests, datetime, math, textwrap
from tqdm import tqdm

FOOTBALL_API_KEY = 'd5f8091f602cb270431726b11d7100cc'
TG_TOKEN        = '8137010387:AAHaZzYPBezbOK7tRbG6w5jcWz3KChSj1E'
CHAT_ID         = '834545801'
HEADERS         = {'x-apisports-key': FOOTBALL_API_KEY}

# ==========================================
# 1. ЗАГРУЗКА ИСТОРИИ
# ==========================================
df_hist = pd.read_csv('/content/BIG_SHKOLA_DATASET.csv')
df_hist = df_hist.dropna(subset=['B365H','B365D','B365A'])
for c in ['FTHG','FTAG']: df_hist[c] = df_hist[c].astype(int)

AVG_HOME = df_hist['FTHG'].mean()
AVG_AWAY = df_hist['FTAG'].mean()
RHO     = -0.15

# ==========================================
# 2. DIXON-COLES ВЕСА (5 итераций)  +  НОРМИРОВКА + КЛИППИНГ
# ==========================================
teams = set(df_hist['home']) | set(df_hist['away'])
# инициализируем «плоско»
attack = {t:1.0 for t in teams}
defence= {t:1.0 for t in teams}
home_adv = 1.0

def clip(x, lo=0.4, hi=3.0):
    return max(lo, min(hi, x))

for it in tqdm(range(5), desc='Dixon-Coles'):
    # E-step
    lam_home = {(r['home'],r['away']): attack[r['home']]*home_adv*defence[r['away']]*AVG_HOME
                for _,r in df_hist.iterrows()}
    lam_away = {(r['home'],r['away']): attack[r['away']]*defence[r['home']]*AVG_AWAY
                for _,r in df_hist.iterrows()}
    # M-step
    for t in teams:
        num_att = df_hist[df_hist['home']==t]['FTHG'].sum() + df_hist[df_hist['away']==t]['FTAG'].sum()
        den_att = sum(lam_home[h,a] for (h,a) in lam_home if h==t) + sum(lam_away[h,a] for (h,a) in lam_away if a==t)
        attack[t] = clip(num_att / (den_att or 1))
        num_def = df_hist[df_hist['home']==t]['FTAG'].sum() + df_hist[df_hist['away']==t]['FTHG'].sum()
        den_def = sum(lam_home[h,a] for (h,a) in lam_home if a==t) + sum(lam_away[h,a] for (h,a) in lam_away if h==t)
        defence[t] = clip(num_def / (den_def or 1))
    home_adv = clip(df_hist['FTHG'].sum() / sum(lam_home.values()), lo=0.8, hi=1.5)

    # нормируем attack/defence к среднему 1.0
    avg_att = sum(attack.values()) / len(attack)
    avg_def = sum(defence.values()) / len(defence)
    for t in teams:
        attack[t] /= avg_att
        defence[t] /= avg_def

# ==========================================
# 3. ОДИН exact-score = максимум плотности + перекос
# ==========================================
def exact_score(lam_h, lam_a, max_goals=4):
    best_p, best_yh, best_ya = 0, 0, 0
    for yh in range(max_goals+1):
        for ya in range(max_goals+1):
            tau = 0.0
            if yh==0 and ya==0: tau = RHO
            if yh==1 and ya==0: tau = -RHO
            if yh==0 and ya==1: tau = -RHO
            if yh==1 and ya==1: tau = RHO
            p = (math.exp(-lam_h)*lam_h**yh/math.factorial(yh) *
                 math.exp(-lam_a)*lam_a**ya/math.factorial(ya) * math.exp(tau))
            if p > best_p:
                best_p, best_yh, best_ya = p, yh, ya
    # перекос фаворита
    if lam_h > lam_a + 1.0:
        best_yh = min(best_yh + 1, max_goals)
    elif lam_a > lam_h + 1.0:
        best_ya = min(best_ya + 1, max_goals)
    return best_yh, best_ya, best_p

# ==========================================
# 4. СЕГОДНЯШНИЕ МАТЧИ
# ==========================================
today = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%d')
resp = requests.get(f'https://v3.football.api-sports.io/fixtures?date={today}', headers=HEADERS, timeout=20).json()
fixtures = resp.get('response', [])
print(f'📅 Матчей на {today}: {len(fixtures)}\n')

# ==========================================
# 5. ОДИН ТОЧНЫЙ СЧЁТ НА КАЖДЫЙ МАТЧ  (λ ≤ 4.0)
# ==========================================
all_preds = []
for f in tqdm(fixtures, desc='Exact-score'):
    home= f['teams']['home']['name']
    away= f['teams']['away']['name']
    if home not in attack or away not in attack:
        continue
    lam_h = attack[home]*home_adv * defence[away] * AVG_HOME
    lam_a = attack[away]          * defence[home] * AVG_AWAY
    # жёстко ограничиваем λ сверху
    lam_h = min(lam_h, 4.0)
    lam_a = min(lam_a, 4.0)
    yh, ya, prob = exact_score(lam_h, lam_a)
    all_preds.append({
        'time': f['fixture']['date'][11:16],
        'league': f"{f['league']['country']} - {f['league']['name']}",
        'match': f'{home} – {away}',
        'score': f'{yh}-{ya}',
        'prob':  f'{prob:.1%}',
        'lambda_h': f'{lam_h:.2f}',
        'lambda_a': f'{lam_a:.2f}'
    })

print(f'🔮 Сформировано {len(all_preds)} точных счётов.\n')

# ---------------------------- КОНСОЛЬ -----------------------------
for v in all_preds:
    print(f"{v['time']}  {v['league']:<25}  {v['match']:<40}  → {v['score']}  (P={v['prob']}, λ={v['lambda_h']}/{v['lambda_a']})")
# ----------------------------------------------------------------

# ==========================================
# 6. TELEGRAM (по желанию, пачками по 5)
# ==========================================
def send(msg):
    url = f'https://api.telegram.org/bot{TG_TOKEN}/sendMessage'
    r = requests.post(url, data={'chat_id': CHAT_ID, 'text': msg, 'parse_mode': 'Markdown'})
    return r.ok

for i in range(0, len(all_preds), 5):
    chunk = all_preds[i:i+5]
    txt = '🎯 **BIG-SCHOOL Consensus Exact-Score**  \n\n'
    for v in chunk:
        txt += f"⏰ {v['time']}  | {v['league']}  \n⚽️ **{v['match']}** → **{v['score']}**  (P={v['prob']}, λ={v['lambda_h']}/{v['lambda_a']})  \n—" + "—"*25 + "\n"
    send(txt)

Dixon-Coles: 100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


📅 Матчей на 2026-01-12: 102



Exact-score: 100%|██████████| 102/102 [00:00<00:00, 212951.22it/s]


🔮 Сформировано 2 точных счётов.

17:30  Italy - Serie A            Genoa – Cagliari                          → 1-0  (P=19.4%, λ=1.35/0.74)
19:45  Italy - Serie A            Juventus – Cremonese                      → 2-0  (P=31.4%, λ=1.39/0.25)


In [ ]:
# ==========================================
# ДОКАЧКА ВСЕГО МИРА (2017-2025)  в  /content/BIG_SHKOLA_DATASET.csv
# ==========================================
import pandas as pd, requests, glob, os

SEASONS = ['1718','1819','1920','2021','2122','2223','2324','2425']
LEAGUES = {   # код football-data : короткое имя
    'E0':'ENG','E1':'ENG-CH','E2':'ENG-L1','E3':'ENG-L2',
    'D1':'GER','D2':'GER-2','I1':'ITA','I2':'ITA-B','SP1':'ESP','SP2':'ESP-2',
    'F1':'FRA','F2':'FRA-2','P1':'POR','N1':'NED','B1':'BEL','A1':'AUT','T1':'TUR',
    'SC0':'SCO','SC1':'SCO-2','SC2':'SCO-3','SC3':'SCO-4',
    'G1':'GRE','E1':'RUS','PL':'POL','CZ':'CZE','SK1':'SVK','H1':'HUN',
    'DK1':'DEN','S1':'SWI','N1':'NOR','S1':'SWE','FI1':'FIN','IS1':'ICE',
    'CL':'UCL','EL':'UEL','EC':'UECL','WC18':'WC-2018','NU21':'EURO-U21','EUQ24':'EURO-Q'
}
BASE_URL = 'https://www.football-data.co.uk/mmz4281/{}/{}.csv'
OLD_FILE = '/content/BIG_SHKOLA_DATASET.csv'
NEW_FILE = '/content/BIG_SHKOLA_FULL.csv'

# старые данные
df_old = pd.read_csv(OLD_FILE) if os.path.exists(OLD_FILE) else pd.DataFrame()
all_df = [df_old]

for yr in SEASONS:
    for code, name in LEAGUES.items():
        url = BASE_URL.format(yr, code)
        try:
            df = pd.read_csv(url)
            df = df[['HomeTeam','AwayTeam','FTHG','FTAG','B365H','B365D','B365A']]
            df.columns = ['home','away','FTHG','FTAG','B365H','B365D','B365A']
            df['league'] = name
            df['season'] = yr
            all_df.append(df)
        except: continue

big = pd.concat(all_df, ignore_index=True)
big = big.dropna(subset=['B365H','B365D','B365A'])
big.to_csv(NEW_FILE, index=False)
print(f'✅ Добавлено {len(big)} матчей → {NEW_FILE}')

✅ Добавлено 75655 матчей → /content/BIG_SHKOLA_FULL.csv


In [ ]:
# ==========================================
# 5. ОДИН ТОЧНЫЙ СЧЁТ НА КАЖДЫЙ МАТЧ  (λ ≤ 4.0)
# ==========================================
all_preds = []
for f in tqdm(fixtures, desc='Exact-score'):
    home= f['teams']['home']['name']
    away= f['teams']['away']['name']
    if home not in attack or away not in attack:
        continue
    lam_h = attack[home]*home_adv * defence[away] * AVG_HOME
    lam_a = attack[away]          * defence[home] * AVG_AWAY
    lam_h = min(lam_h, 4.0)
    lam_a = min(lam_a, 4.0)
    yh, ya, prob = exact_score(lam_h, lam_a)
    all_preds.append({
        'time': f['fixture']['date'][11:16],
        'league': f"{f['league']['country']} - {f['league']['name']}",
        'match': f'{home} – {away}',
        'score': f'{yh}-{ya}',
        'prob':  f'{prob:.1%}',
        'lambda_h': f'{lam_h:.2f}',
        'lambda_a': f'{lam_a:.2f}'
    })

print(f'🔮 Сформировано {len(all_preds)} точных счётов.\n')

# ---------------------------- КОНСОЛЬ -----------------------------
for v in all_preds:
    print(f"{v['time']}  {v['league']:<25}  {v['match']:<40}  → {v['score']}  (P={v['prob']}, λ={v['lambda_h']}/{v['lambda_a']})")
# ----------------------------------------------------------------

Exact-score: 100%|██████████| 102/102 [00:00<00:00, 252251.77it/s]

🔮 Сформировано 2 точных счётов.

17:30  Italy - Serie A            Genoa – Cagliari                          → 1-0  (P=19.4%, λ=1.35/0.74)
19:45  Italy - Serie A            Juventus – Cremonese                      → 2-0  (P=31.4%, λ=1.39/0.25)


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


In [ ]:
# ==========================================
# 0. УСТАНОВКИ И БИБЛИОТЕКИ
# ==========================================
import pandas as pd
import numpy as np
import requests
import datetime
import math
from tqdm import tqdm
import catboost as cb

# --- КОНФИГУРАЦИЯ ---
FOOTBALL_API_KEY = 'd5f8091f602cb270431726b11d7100cc'
TG_TOKEN         = '8137010387:AAHaZzYPBezbOK7tRbG6vw5jcWz3KChSj1E'
CHAT_ID          = '834545801'
HEADERS          = {'x-apisports-key': FOOTBALL_API_KEY}
CSV_PATH         = '/content/BIG_SHKOLA_FULL.csv'

# Имена колонок кэфов
COL_HOME = 'B365H'
COL_DRAW = 'B365D'
COL_AWAY = 'B365A'
# Попытаемся найти колонку тотала, если нет — создадим заглушку
COL_TOTAL_OVER_GUESS = ['B365>2.5', 'TO2.5', 'BbAv>2.5', 'Avg>2.5']

# ==========================================
# 1. ЗАГРУЗКА И ЧИСТКА (БРОНЕБОЙНАЯ)
# ==========================================
print(f"📂 Загружаем базу: {CSV_PATH}...")

# low_memory=False убирает DtypeWarning
df_hist = pd.read_csv(CSV_PATH, low_memory=False)

# 1. Ищем колонку тотала или создаем заглушку
real_total_col = None
for col in COL_TOTAL_OVER_GUESS:
    if col in df_hist.columns:
        real_total_col = col
        break

if real_total_col:
    COL_TOTAL_OVER = real_total_col
    print(f"✅ Найдена колонка тотала: {COL_TOTAL_OVER}")
else:
    print("⚠️ Колонки тотала нет. Создаем фиктивную (1.90) для работы модели.")
    COL_TOTAL_OVER = 'Manual_O2.5'
    df_hist[COL_TOTAL_OVER] = 1.90

# 2. ПРИНУДИТЕЛЬНАЯ ЧИСТКА СЧЕТА И КЭФОВ
# Превращаем все не-числа в NaN (ошибки парсинга, пустые строки)
cols_to_clean = ['FTHG', 'FTAG', COL_HOME, COL_DRAW, COL_AWAY, COL_TOTAL_OVER]
for c in cols_to_clean:
    df_hist[c] = pd.to_numeric(df_hist[c], errors='coerce')

# 3. Удаляем строки, где хоть что-то важное равно NaN
before_drop = len(df_hist)
df_hist = df_hist.dropna(subset=cols_to_clean)
print(f"🧹 Удалено битых строк: {before_drop - len(df_hist)}")

# 4. Теперь безопасно конвертируем голы в int
df_hist['FTHG'] = df_hist['FTHG'].astype(int)
df_hist['FTAG'] = df_hist['FTAG'].astype(int)

# 5. Формируем X и y
X = df_hist[[COL_HOME, COL_DRAW, COL_AWAY, COL_TOTAL_OVER]].values
y_home = df_hist['FTHG'].values
y_away = df_hist['FTAG'].values

print(f"✅ База готова: {len(df_hist)} чистых матчей.")

# ==========================================
# 2. ОБУЧЕНИЕ CATBOOST
# ==========================================
print("🧠 Обучение нейросети (CatBoost)...")
# verbose=0 чтобы не спамило в консоль
model_home = cb.CatBoostRegressor(depth=6, iterations=500, learning_rate=0.05, verbose=0)
model_away = cb.CatBoostRegressor(depth=6, iterations=500, learning_rate=0.05, verbose=0)

model_home.fit(X, y_home)
model_away.fit(X, y_away)
print('✅ Модели обучены.')

# ==========================================
# 3. POISSON С КОРРЕКЦИЕЙ
# ==========================================
def poisson_predict(lambda_home, lambda_away, max_goals=5):
    best_p, best_yh, best_ya = 0, 0, 0
    for yh in range(max_goals+1):
        for ya in range(max_goals+1):
            p = (math.exp(-lambda_home) * lambda_home**yh / math.factorial(yh) *
                 math.exp(-lambda_away) * lambda_away**ya / math.factorial(ya))
            # Коррекция ничьих
            if yh == ya:
                if yh == 0: p *= 1.20
                if yh == 1: p *= 1.10
            if p > best_p:
                best_p, best_yh, best_ya = p, yh, ya
    return best_yh, best_ya, best_p

# ==========================================
# 4. ПАРСИНГ И ПРОГНОЗ
# ==========================================
today = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%d')
print(f"\n📡 Запрос матчей на {today}...")
try:
    resp = requests.get(f'https://v3.football.api-sports.io/fixtures?date={today}', headers=HEADERS, timeout=20).json()
    fixtures = resp.get('response', [])
except Exception as e:
    print(f"❌ Ошибка API: {e}")
    fixtures = []

print(f'📅 Найдено матчей: {len(fixtures)}')
ns_fixtures = [f for f in fixtures if f['fixture']['status']['short'] == 'NS']
all_preds = []

for f in tqdm(ns_fixtures, desc='Анализ'):
    fid   = f['fixture']['id']
    home  = f['teams']['home']['name']
    away  = f['teams']['away']['name']

    try:
        o_res = requests.get(f"https://v3.football.api-sports.io/odds?fixture={fid}", headers=HEADERS).json()
        if not o_res['response']: continue
        bookmakers = o_res['response'][0]['bookmakers']
        if not bookmakers: continue

        bookie = next((b for b in bookmakers if b['id'] == 1 or b['name'] == 'Bet365'), bookmakers[0])
        bets = bookie['bets']

        # 1X2
        bet_1x2 = next((b for b in bets if b['id'] == 1), None)
        if not bet_1x2: continue
        vals = bet_1x2['values']
        h_o = float(next(v['odd'] for v in vals if v['value'] == 'Home'))
        d_o = float(next(v['odd'] for v in vals if v['value'] == 'Draw'))
        a_o = float(next(v['odd'] for v in vals if v['value'] == 'Away'))

        # Total Over 2.5
        tb_o = 1.90 # Дефолт
        bet_ou = next((b for b in bets if b['id'] == 5), None)
        if bet_ou:
            try:
                tb_val = next(v for v in bet_ou['values'] if v['value'] == 'Over 2.5')
                tb_o = float(tb_val['odd'])
            except: pass

        # Прогноз
        X_new = np.array([[h_o, d_o, a_o, tb_o]])
        lh = max(0.1, model_home.predict(X_new)[0])
        la = max(0.1, model_away.predict(X_new)[0])
        yh, ya, prob = poisson_predict(lh, la)

        all_preds.append({
            'time': f['fixture']['date'][11:16],
            'league': f"{f['league']['country']} | {f['league']['name']}",
            'match': f'{home} vs {away}',
            'score': f'{yh}-{ya}',
            'prob':  prob,
            'desc':  f"λ: {lh:.2f}/{la:.2f}",
            'odds':  f"{h_o}|{d_o}|{a_o}"
        })

    except: continue

# ==========================================
# 5. ОТПРАВКА
# ==========================================
all_preds.sort(key=lambda x: x['prob'], reverse=True)
print(f'\n🔮 Готово прогнозов: {len(all_preds)}')

for i in range(0, len(all_preds), 5):
    chunk = all_preds[i:i+5]
    txt = f"🎯 **SHKOLA | EXACT SCORE**\n"
    for v in chunk:
        icon = "🔥" if v['prob'] > 0.13 else "📊"
        txt += f"⏰ {v['time']} {v['league']}\n⚽️ {v['match']} -> **{v['score']}** {icon} ({v['prob']:.1%})\n—\n"

    requests.post(f'https://api.telegram.org/bot{TG_TOKEN}/sendMessage',
                  data={'chat_id': CHAT_ID, 'text': txt, 'parse_mode': 'Markdown'})
    print(f"Отправлена пачка {i//5 + 1}")

📂 Загружаем базу: /content/BIG_SHKOLA_FULL.csv...
⚠️ Колонки тотала нет. Создаем фиктивную (1.90) для работы модели.
🧹 Удалено битых строк: 1
✅ База готова: 75654 чистых матчей.
🧠 Обучение нейросети (CatBoost)...
✅ Модели обучены.

📡 Запрос матчей на 2026-01-12...
📅 Найдено матчей: 102


Анализ: 100%|██████████| 71/71 [00:12<00:00,  5.63it/s]



🔮 Готово прогнозов: 53
Отправлена пачка 1
Отправлена пачка 2
Отправлена пачка 3
Отправлена пачка 4
Отправлена пачка 5
Отправлена пачка 6
Отправлена пачка 7
Отправлена пачка 8
Отправлена пачка 9
Отправлена пачка 10
Отправлена пачка 11


In [ ]:
# ==========================================
# 0. УСТАНОВКИ И БИБЛИОТЕКИ
# ==========================================
# Если запускаешь в Colab первый раз, раскомментируй строку ниже:
# !pip install catboost

import pandas as pd
import numpy as np
import requests
import datetime
import math
from tqdm import tqdm
import catboost as cb

# --- КОНФИГУРАЦИЯ ---
FOOTBALL_API_KEY = 'd5f8091f602cb270431726b11d7100cc'
# ИСПРАВЛЕННЫЙ ТОКЕН (с буквой v):
TG_TOKEN         = '8137010387:AAHaZzYPBezbOK7tRbG6vw5jcWz3KChSj1E'
CHAT_ID          = '834545801'
HEADERS          = {'x-apisports-key': FOOTBALL_API_KEY}
CSV_PATH         = '/content/BIG_SHKOLA_FULL.csv'

# Настройки поиска колонок в CSV
COL_HOME = 'B365H'
COL_DRAW = 'B365D'
COL_AWAY = 'B365A'
COL_TOTAL_OVER_GUESS = ['B365>2.5', 'TO2.5', 'BbAv>2.5', 'Avg>2.5']

# Настройки SMART-фильтра
SKIP_KEYWORDS = ['U19', 'U20', 'U21', 'U23', 'Youth', 'Reserve', 'Women', 'Feminin', 'Friendlies']
GOLD_LEAGUES  = ['Iran', 'Egypt', 'Morocco', 'Greece', 'Spain: Segunda', 'Italy: Serie C', 'Argentina', 'France: Ligue 2']

# ==========================================
# 1. ЗАГРУЗКА И ЧИСТКА БАЗЫ
# ==========================================
print(f"📂 Загружаем базу: {CSV_PATH}...")

try:
    df_hist = pd.read_csv(CSV_PATH, low_memory=False)

    # 1. Ищем колонку тотала
    real_total_col = None
    for col in COL_TOTAL_OVER_GUESS:
        if col in df_hist.columns:
            real_total_col = col
            break

    if real_total_col:
        COL_TOTAL_OVER = real_total_col
        print(f"✅ Найдена колонка тотала: {COL_TOTAL_OVER}")
    else:
        print("⚠️ Колонки тотала нет. Создаем заглушку (1.90).")
        COL_TOTAL_OVER = 'Manual_O2.5'
        df_hist[COL_TOTAL_OVER] = 1.90

    # 2. Чистка мусора (превращаем ошибки в NaN)
    cols_to_clean = ['FTHG', 'FTAG', COL_HOME, COL_DRAW, COL_AWAY, COL_TOTAL_OVER]
    for c in cols_to_clean:
        df_hist[c] = pd.to_numeric(df_hist[c], errors='coerce')

    # 3. Удаляем битые строки
    df_hist = df_hist.dropna(subset=cols_to_clean)

    # 4. Формируем данные для обучения
    df_hist['FTHG'] = df_hist['FTHG'].astype(int)
    df_hist['FTAG'] = df_hist['FTAG'].astype(int)

    X = df_hist[[COL_HOME, COL_DRAW, COL_AWAY, COL_TOTAL_OVER]].values
    y_home = df_hist['FTHG'].values
    y_away = df_hist['FTAG'].values

    print(f"✅ База готова: {len(df_hist)} чистых матчей.")

except Exception as e:
    print(f"❌ Критическая ошибка загрузки CSV: {e}")
    exit()

# ==========================================
# 2. ОБУЧЕНИЕ МОДЕЛЕЙ (CatBoost)
# ==========================================
print("🧠 Обучение нейросетей (Home & Away)...")
model_home = cb.CatBoostRegressor(depth=6, iterations=500, learning_rate=0.05, verbose=0)
model_away = cb.CatBoostRegressor(depth=6, iterations=500, learning_rate=0.05, verbose=0)

model_home.fit(X, y_home)
model_away.fit(X, y_away)
print('✅ Модели готовы к работе.')

# ==========================================
# 3. ФУНКЦИЯ ПРОГНОЗА (POISSON SMART)
# ==========================================
def poisson_predict(lambda_home, lambda_away, max_goals=5):
    best_p, best_yh, best_ya = 0, 0, 0
    for yh in range(max_goals+1):
        for ya in range(max_goals+1):
            p = (math.exp(-lambda_home) * lambda_home**yh / math.factorial(yh) *
                 math.exp(-lambda_away) * lambda_away**ya / math.factorial(ya))

            # Коррекция на ничью (Anti-Bias)
            if yh == ya:
                if yh == 0: p *= 1.20 # Бонус для 0-0
                if yh == 1: p *= 1.10 # Бонус для 1-1

            if p > best_p:
                best_p, best_yh, best_ya = p, yh, ya
    return best_yh, best_ya, best_p

# ==========================================
# 4. ПАРСИНГ И ФИЛЬТРАЦИЯ
# ==========================================
today = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%d')
print(f"\n📡 Запрос матчей на {today}...")

try:
    resp = requests.get(f'https://v3.football.api-sports.io/fixtures?date={today}', headers=HEADERS, timeout=20).json()
    fixtures = resp.get('response', [])
except Exception as e:
    print(f"❌ Ошибка API: {e}")
    fixtures = []

print(f'📅 Всего матчей в линии: {len(fixtures)}')
ns_fixtures = [f for f in fixtures if f['fixture']['status']['short'] == 'NS']

all_preds = []

print("⚙️ Анализ и фильтрация...")
for f in tqdm(ns_fixtures):
    # --- БЛОК ФИЛЬТРАЦИИ ---
    league_name = f['league']['name']
    country = f['league']['country']
    full_league = f"{country}: {league_name}"

    # 1. Выкидываем мусор (Молодежки, Женщины, Товарняки)
    if any(k in league_name for k in SKIP_KEYWORDS) or any(k in country for k in SKIP_KEYWORDS):
        continue

    fid   = f['fixture']['id']
    home  = f['teams']['home']['name']
    away  = f['teams']['away']['name']

    try:
        # Тянем кэфы
        o_res = requests.get(f"https://v3.football.api-sports.io/odds?fixture={fid}", headers=HEADERS).json()
        if not o_res['response']: continue
        bookmakers = o_res['response'][0]['bookmakers']
        if not bookmakers: continue

        bookie = next((b for b in bookmakers if b['id'] == 1 or b['name'] == 'Bet365'), bookmakers[0])
        bets = bookie['bets']

        # 1. 1X2 (Winner)
        bet_1x2 = next((b for b in bets if b['id'] == 1), None)
        if not bet_1x2: continue
        vals = bet_1x2['values']
        h_o = float(next(v['odd'] for v in vals if v['value'] == 'Home'))
        d_o = float(next(v['odd'] for v in vals if v['value'] == 'Draw'))
        a_o = float(next(v['odd'] for v in vals if v['value'] == 'Away'))

        # 2. Total Over 2.5 (Goals)
        tb_o = 1.90 # Дефолт если нет в линии
        bet_ou = next((b for b in bets if b['id'] == 5), None)
        if bet_ou:
            try:
                tb_val = next(v for v in bet_ou['values'] if v['value'] == 'Over 2.5')
                tb_o = float(tb_val['odd'])
            except: pass

        # --- ПРОГНОЗИРОВАНИЕ ---
        X_new = np.array([[h_o, d_o, a_o, tb_o]])
        lh = max(0.1, model_home.predict(X_new)[0])
        la = max(0.1, model_away.predict(X_new)[0])
        yh, ya, prob = poisson_predict(lh, la)

        # --- ОЦЕНКА КАЧЕСТВА (ICONS) ---
        zone_icon = "⚠️"
        zone_text = "Risk"

        # Зеленая зона: Золотая лига + Низовой прогноз
        is_gold_league = any(gl in full_league for gl in GOLD_LEAGUES)
        if is_gold_league and (yh+ya) <= 2:
            zone_icon = "✅"
            zone_text = "SMART (Defensive)"

        # Алмаз: Очень высокая вероятность
        if prob > 0.15:
            zone_icon = "💎"
            zone_text = "DIAMOND (High Prob)"

        all_preds.append({
            'time': f['fixture']['date'][11:16],
            'league': full_league,
            'match': f'{home} vs {away}',
            'score': f'{yh}-{ya}',
            'prob':  prob,
            'desc':  zone_text,
            'icon':  zone_icon
        })

    except: continue

# ==========================================
# 5. ОТПРАВКА В TELEGRAM (ТОЛЬКО ЭЛИТА)
# ==========================================
all_preds.sort(key=lambda x: x['prob'], reverse=True)

# Берем ТОП-15 лучших
top_preds = all_preds[:15]

print(f'\n🔮 Отобрано {len(top_preds)} лучших прогнозов (мусор отсеян).')

if not top_preds:
    print("❌ Нет подходящих матчей после фильтрации.")
else:
    for i in range(0, len(top_preds), 5):
        chunk = top_preds[i:i+5]
        txt = f"🎯 **SHKOLA | SMART FILTER**\n"
        for v in chunk:
            txt += (
                f"{v['icon']} **{v['score']}** ({v['prob']:.1%})\n"
                f"⚽️ {v['match']}\n"
                f"🏆 {v['league']}\n"
                f"🔍 _{v['desc']}_\n"
                f"—\n"
            )

        # Отправка
        url = f'https://api.telegram.org/bot{TG_TOKEN}/sendMessage'
        params = {'chat_id': CHAT_ID, 'text': txt, 'parse_mode': 'Markdown'}
        try:
            r = requests.post(url, data=params)
            if r.ok:
                print(f"📤 Пачка {i//5 + 1} успешно отправлена!")
            else:
                print(f"❌ Ошибка отправки в ТГ: {r.text}")
        except Exception as e:
            print(f"❌ Ошибка соединения: {e}")

print("\n🏁 ЗАВЕРШЕНО.")

📂 Загружаем базу: /content/BIG_SHKOLA_FULL.csv...
⚠️ Колонки тотала нет. Создаем заглушку (1.90).
✅ База готова: 75654 чистых матчей.
🧠 Обучение нейросетей (Home & Away)...
✅ Модели готовы к работе.

📡 Запрос матчей на 2026-01-12...
📅 Всего матчей в линии: 102
⚙️ Анализ и фильтрация...


100%|██████████| 70/70 [00:09<00:00,  7.73it/s]



🔮 Отобрано 15 лучших прогнозов (мусор отсеян).
📤 Пачка 1 успешно отправлена!
📤 Пачка 2 успешно отправлена!
📤 Пачка 3 успешно отправлена!

🏁 ЗАВЕРШЕНО.
